In [ ]:
#pip install ipynb
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from gensim.sklearn_api.phrases import PhrasesTransformer

In [ ]:
from ipynb.fs.full.pipeline_class import Text_preprocessor, Text_preprocessor_sentence_tokenize, Text_extrafeatures, func_dict, TfidfEmbeddingVectorizer, CollocationGenerator

In [ ]:
#PRECONDICION, LOS DATOS SE OBTIENEN DE dataframe.pkl QUE TIENE LOS SUBTITULOS YA LIMPIOS 
#Creacion de Pipe de preprocesamiento
def dataset_pipeline(LSA_K = 10,LDA_TOPICS = 10,max_df_tfidf=0.9, min_df_tfidf=0.1,
                    max_df_cv=0.9 , min_df_cv=0.1):
    #%%time
    #LSA_K -- cantidad de "topicos" LSA
    #LDA_TOPICS -- cantidad de topicos LDA

    tf_idf_vect = TfidfVectorizer(max_df_tfidf, min_df_tfidf, stop_words="english" )
    tf_idf_transf = TfidfTransformer() #trasnforma el count_vectorizer en tfidf, unsando el transformador, me evito volver a calcularlo
    count_vect = CountVectorizer(max_df_cv, min_df_cv, stop_words="english")
    new_features = Text_extrafeatures(func_dict) # func_dict es para alacenar funciones que crean nuevos features
    SVD = TruncatedSVD(n_components=LSA_K, algorithm = 'arpack')
    lda = LatentDirichletAllocation(n_components=LDA_TOPICS, max_iter=100, learning_method="online", n_jobs=-1)
    w2v = TfidfEmbeddingVectorizer(tf_idf_vect)
    lemma = Text_preprocessor()
    sent_token = Text_preprocessor_sentence_tokenize()
    collocations = CollocationGenerator()
    sw = Text_preprocessor_lemmatizer()
    
    text_preproc = Pipeline([("lemmatizer",lemma), 
                             ("sent_token",sent_token),
                             ("collocations",collocations),
                             ("sw",sw)
                            ])

    lsa_tfidf_pipe = Pipeline([("tfidf_transf", tf_idf_transf),
                                ("lsa",SVD)
                               ])

    topic_pipe = Pipeline([
                            ("tf_vec", count_vect),
                            ("topic_features",FeatureUnion([
                                                            ("lda",lda),
                                                            ("lsa_tfidf_pipe",lsa_tfidf_pipe),
                                                            ]))
                        ])


    new_features_pipe = Pipeline([
                                    ("new_features",new_features),
                                    ("std_scaler",StandardScaler())
                               ])

#Se comenta text preproc y w2v porque llevaban mucho tiempo de ejecución
    dataset_pipeline = Pipeline([#("text_preproc", text_preproc),
                                 ("text_features",FeatureUnion([
                                                                ("new_features",new_features_pipe),
                                                                ("topics",topic_pipe),
                                                                #("embedding",w2v)
                                                                ]))


                                ])

    return dataset_pipeline

In [ ]:
#dataset_result= dataset_pipeline().fit_transform(X_train)
#feature_names = new_features.get_feature_names() +["LSA_%d"%(k+1) for k in np.arange(LSA_K)] + ["LDA_%d"%(k+1) for k in np.arange(LDA_TOPICS)] + tf_idf_vect.get_feature_names()
#dataset_result_df = DataFrame(dataset_result.todense(), columns=feature_names, index = X_train.index)